In [1]:
import os
import requests
from bs4 import BeautifulSoup

In [2]:
def clean_naver_blog_url(blog_url):
    """
    네이버 블로그는 외부 링크를 통해 redirect 되는 경우가 많음.
    원래 포스트 url을 iframe으로 추적하는 구조로 구성되어 있음.
    """
    if "blog.naver.com" in blog_url:
        return blog_url
    else:
        # 통합 검색 결과 redirect 우회
        res = requests.get(blog_url, allow_redirects=True)
        return res.url

def fetch_blog_content(blog_url):
    cleaned_url = clean_naver_blog_url(blog_url)
    res = requests.get(cleaned_url)
    soup = BeautifulSoup(res.text, "html.parser")

    # 네이버 블로그 구조: iframe 안에 실제 콘텐츠 존재
    iframe = soup.find("iframe", {"id": "mainFrame"})
    if iframe and "src" in iframe.attrs:
        iframe_url = "https://blog.naver.com" + iframe["src"]
        iframe_res = requests.get(iframe_url)
        iframe_soup = BeautifulSoup(iframe_res.text, "html.parser")

        # 본문 텍스트 추출
        content_div = iframe_soup.find("div", {"class": "se-main-container"})
        if content_div:
            return content_div.get_text(strip=True)
        else:
            return "[본문 추출 실패: 구조 다름]"
    else:
        return "[본문 없음 또는 iframe 구조 아님]"

In [3]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote
from dotenv import load_dotenv

load_dotenv()
NAVER_CLIENT_ID = os.getenv("NAVER_CLIENT_ID")
NAVER_CLIENT_SECRET = os.getenv("NAVER_CLIENT_SECRET")

def search_news(query, display=5):
    url = "https://openapi.naver.com/v1/search/news.json"
    headers = {
        "X-Naver-Client-Id": NAVER_CLIENT_ID,
        "X-Naver-Client-Secret": NAVER_CLIENT_SECRET
    }
    params = {
        "query": query,
        "display": display,
        "start": 1,
        "sort": "sim"
    }

    res = requests.get(url, headers=headers, params=params)
    res.raise_for_status()
    return res.json()["items"]

def clean_news_url(url):
    """
    외부 프록시/리다이렉트 URL 제거
    """
    if "n.news.naver.com" in url:
        return url
    try:
        res = requests.get(url, allow_redirects=True, timeout=5)
        return res.url
    except:
        return url

def fetch_news_content(news_url):
    clean_url = clean_news_url(news_url)
    res = requests.get(clean_url, headers={"User-Agent": "Mozilla/5.0"})
    soup = BeautifulSoup(res.text, "html.parser")

    # 본문 영역 클래스 (일반 뉴스/연예/스포츠 모두 다름)
    candidates = [
        {"id": "dic_area"},                             # 일반 기사
        {"id": "articeBody"},                           # 스포츠/연예
        {"class_": "newsct_article"},                   # 모바일 버전
        {"class_": "article_body"},                     # 예외적 케이스
    ]

    for c in candidates:
        content = soup.find("div", **c)
        if content:
            return content.get_text(strip=True)

    return "[본문 추출 실패]"

# 실행 예제
if __name__ == "__main__":
    query = "삼성전자"
    items = search_news(query)

    for item in items:
        print("제목:", item["title"])
        print("URL:", item["link"])
        content = fetch_news_content(item["link"])
        print("본문:", content[:300], "...")
        print("=" * 80)


제목: <b>삼성전자</b>, 세계 각지서 '삼성 비전 AI' 옥외광고 선봬
URL: https://n.news.naver.com/mnews/article/003/0013235130?sid=101
본문: 올해 말까지 뉴욕·런던 등서 옥외광고[서울=뉴시스]미국 뉴욕 타임스 스퀘어에서 진행하는 '삼성 비전 AI(Vision AI)' 옥외광고 모습. (사진=삼성전자 제공) 2025.05.12. photo@newsis.com *재판매 및 DB 금지[서울=뉴시스]이지용 기자 = 삼성전자가 글로벌 주요 랜드마크에서 '삼성 비전 AI' 옥외광고를 선보인다.삼성전자는 미국 뉴욕 타임스 스퀘어, 영국 런던 피카딜리 광장, 중국 상해 홍이 인터내셔널 플라자 등에서 삼성 비전 AI 옥외광고를 올해 말까지 진행한다고 12일 밝혔다.삼성 비전 AI는 사용 ...
제목: <b>삼성전자</b>, 전 세계 랜드마크서 '삼성 비전 AI' 옥외광고
URL: https://n.news.naver.com/mnews/article/001/0015381699?sid=101
본문: 삼성전자 '비전 AI' 옥외광고(서울=연합뉴스) 삼성전자가 영국 런던 피카딜리 광장에서 진행하는 '삼성 비전 AI(Vision AI)' 옥외광고 모습. 2025.5.12 [삼성전자 제공. 재판매 및 DB 금지](서울=연합뉴스) 강태우 기자 = 삼성전자가 글로벌 주요 랜드마크에서 '삼성 비전 AI'(Vision AI) 옥외광고를 통해 'AI TV=삼성' 입지를 굳히고 있다.삼성전자는 미국 뉴욕 타임스 스퀘어, 영국 런던 피카딜리 광장, 중국 상해 홍이 인터내셔널 플라자 등에서 올해 말까지 삼성 비전 AI 옥외광고를 진행한다고 12일  ...
제목: <b>삼성전자</b>-TSMC 반도체 매출 10조 이상 벌어져
URL: https://n.news.naver.com/mnews/article/374/0000439834?sid=101
본문: 삼성전자와 대만 TSMC의 올해 1분기 반도체 매출 격차가 더 크게 벌어진 것으로 집계됐습니

In [4]:
item

{'title': "[기업] <b>삼성전자</b>, 뉴욕·런던 등 주요 랜드마크 '비전 AI' 옥외광고",
 'originallink': 'https://www.ytn.co.kr/_ln/0102_202505121553251219',
 'link': 'https://n.news.naver.com/mnews/article/052/0002191960?sid=101',
 'description': "<b>삼성전자</b>가 올해 말까지 세계 주요 랜드마크에서 '비전 AI' 옥외광고를 선보입니다. <b>삼성전자</b>는 뉴욕 타임스스퀘어와 런던 피카딜리 서커스 등 주요 랜드마크 전광판 광고를 통해 소비자와의 접점을 확대할 계획이라고... ",
 'pubDate': 'Mon, 12 May 2025 15:53:00 +0900'}